# Explainable AI (XAI) Demo: Credit Risk Assessment

This notebook demonstrates how to use SHAP and LIME to interpret a `RandomForestClassifier` trained on a synthetic credit scoring dataset.

**Objective:** Understand which features are most influential in the model's decision to approve or deny a loan.

**Workflow:**
1. **Load Data:** Import the credit data using our custom function.
2. **Train Model:** Train a RandomForest model using our modular training script.
3. **Explain with SHAP:** Generate a global summary plot to see overall feature importance.
4. **Explain with LIME:** Explain a single prediction to understand why one specific applicant was approved or denied.

In [ ]:
import pandas as pd
import sys
import shap
import matplotlib.pyplot as plt

# Add the src directory to the Python path to import our modules
sys.path.append('../src')

from data_preprocessing import load_data
from model_training import train_random_forest_classifier
from explainability import get_shap_explainer, get_lime_explainer

### 1. Load and Prepare Data

In [ ]:
df = load_data('../data/credit_data.csv')

# Define features (X) and target (y)
X = df.drop('target', axis=1)
y = df['target']

X.head()

### 2. Train the Classification Model

In [ ]:
model, X_test, y_test = train_random_forest_classifier(X, y)

### 3. Explain the Model with SHAP (Global Explanation)

SHAP provides a powerful way to understand the overall importance of each feature. The summary plot below shows how much each feature contributes to the model's output.

In [ ]:
explainer, shap_values = get_shap_explainer(model, X_test)

# Generate the SHAP summary plot
shap.summary_plot(shap_values, X_test, plot_type="bar")

### 4. Explain a Single Prediction with LIME (Local Explanation)

Now, let's pick a single instance from our test set and see why the model made the prediction it did for that specific case.

In [ ]:
lime_explainer = get_lime_explainer(X, class_names=['Denied', 'Approved'])

# Choose a single instance to explain (e.g., the 5th instance in the test set)
instance_to_explain = X_test.iloc[[5]]
instance_index = 5

print(f"Model Prediction for instance {instance_index}: {'Approved' if model.predict(instance_to_explain)[0] == 1 else 'Denied'}")
print(f"Actual Label: {'Approved' if y_test.iloc[instance_index] == 1 else 'Denied'}")

# Generate the LIME explanation
explanation = lime_explainer.explain_instance(
    data_row=instance_to_explain.values[0],
    predict_fn=model.predict_proba,
    num_features=5
)

explanation.show_in_notebook(show_table=True)